In [1]:
import random
import torch
import torch.nn as nn

In [11]:
embed = nn.Embedd

tensor([[1, 2],
        [4, 5]])

In [9]:
test = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(test)
test[:, :2] = test[:, :2] + torch.tensor([[1, 2], [4, 5]])
print(test)

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[ 2,  4,  3],
        [ 8, 10,  6]])
